# Workshop II - Swarm Intelligence and Sinergy: Ant Colony for the Traveling Salesman Problem

## Workshop Definition:

In this second workshop, the idea is you solve the _Traveling Salesman Problem (TSP)_ using _Ant Colony Optimization (ACO)_ algorithm. It is time to apply a very popular swarm intelligence algorithm to a problem in the real world.

Imagine you have been hired as __artificial intelligence engineer__ in an important logistic company. Your boss, a Chief Technology Officer, wants to get _shortest path_ to deliver some products. You analyze the problem, and figure out it is similar to _TSP_, and you remember you know some _swarm intellinge_ algoritms useful to solve this problem, in this case, _ACO_.

Here you will have some tasks in order to complete this _workshop_:

1. Create some diagrams and explanations to represent/understand the problem following a _systems thinking approach_.
1. Generate several random $3D$ space points in order to define cities to be visited by the salesman.
1. With random points, generate a list in order to define the requirements for the route.
1. Implement ACO in order to solve TSP problem. Test different parameters combination.
1. Draw a $3D$ plot in order to see the output of the algorithm.
1. Think some conclusions based on outputs analysis.

Write any technical concern/decision/difficulty  you think is relevant regarding your work. You must deliver a full report detailing each one of the previous steps. I strongly recommend you to use a _Jupyter Notebook_ or a _COLAB_ to write/execute your code. Also, your work must include the __report__, the __code__, and a __README.md__ file in repo of the course, everything in a folder called _workshop\_2_.

## Traveling Salesman Problem

Traveling Salesman Problem (TSP) is a pretty common combinatorial optimization problem. The general idea is a salesman must to visit a set of cities exactly once and return to thr original city. The goal is to get the shortest possible route starting on a specific city.

Thinks in all possible combinations, specially when the number of cities growth, it becomes this problem a _NP-Hard_.

In [ ]:
import numpy as np


def generate_cities(number_cities: int) -> list:
    """
    This function generates a list of cities with random coordinates in 3D space.

    Parameters:
    - number_cities (int): Number of cities to generate.

    Returns:
    - list: A list of cities with random coordinates.
    """
    # HERE Generate random 3D Points using numpy generator
    cities = []
    for _ in range(number_cities):
        # Generate random coordinates for each city
        x, y, z = np.random.rand(3)
        np.array(cities.append(np.array([x, y, z])))
    return cities



def calculate_distance(point_1: np.array, point_2: np.array) -> float:
    """
    This function calculates the Euclidean distance between two points.

    Parameters:
    - point_1 (np.array): First point.
    - point_2 (np.array): Second point.

    Returns:
    - float: The Euclidean distance between the two points.
    """
    # HERE return distance between two points using euclidean distance formula
    distance = np.linalg.norm(point_1 - point_2)
    return distance

generate_cities_result = generate_cities(5)

#for city in generate_cities_result:
#    print(city)

distance_cities = np.zeros((len(generate_cities_result), len(generate_cities_result)))
for i in range(len(generate_cities_result)):
    for j in range(len(generate_cities_result)):
        distance_cities[i, j] = calculate_distance(generate_cities_result[i], generate_cities_result[j])
print(distance_cities)

number = len(generate_cities_result)
print(number)
pheromone = np.ones((number, number))
# HERE Initialize pheromone matrix with ones
print(pheromone)

## Ant Colony Optimization (ACO)

Ant Colony Optimization is an algorithm based on ants behaviors to find routes to food using pheromones as a guide of the best way. This is an algorithm simple to implement but get incredible results in route optimization problems.

The idea is the way with most pheromone t is the better way to accomplished the route.

In [ ]:
def ant_colony_optimization(
    cities, n_ants, n_iterations, alpha, beta, evaporation_rate, Q
):
    """
    This function solves the Traveling Salesman Problem using Ant Colony Optimization.

    Parameters:
    - cities (list): List of cities.
    - n_ants (int): Number of ants.
    - n_iterations (int): Number of iterations.
    - alpha (float): It determines how much the ants are influenced by the pheromone trails left by other ants.
    - beta (float):  It determines how much the ants are influenced by the distance to the next city
    - evaporation_rate (float): Evaporation rate.
    - Q (float): It determines the intensity of the pheromone trail left behind by an ant.
    """

    number_cities =  len(cities) # HERE Get number of points
    pheromone = np.ones((number_cities, number_cities))
    # HERE Initialize pheromone matrix with ones

    # initialize output metrics
    best_path = None
    best_path_length = np.inf

    # per each iteration the ants will build a path
    for iteration in range(n_iterations):
        paths = []  # store the paths of each ant
        path_lengths = []

        for ant in range(n_ants):
            visited = [False] * number_cities

            # you could start from any city, but let's start from a random one
            current_city = np.random.randint(number_cities)
            visited[current_city] = True
            path = [current_city]
            path_length = 0

            while False in visited:  # while there are unvisited cities
                unvisited = np.where(np.logical_not(visited))[0]
                probabilities = np.zeros(len(unvisited))

                # based on pheromone, distance and alpha and beta parameters, define the preference
                # for an ant to move to a city
                for i, unvisited_city in enumerate(unvisited):
                    # HERE add equation to calculate the probability of moving to a city based on pheromone, distance and alpha and beta parameters
                    pheromone_factor = pheromone[current_city, unvisited_city] ** alpha
                    distance_factor = 1.0 / calculate_distance(cities[current_city], cities[unvisited_city]) ** beta
                    probabilities[i] = pheromone_factor * distance_factor
                # normalize probabilities, it means, the sum of all probabilities is 1
                # HERE add normalization for calculated probabilities
                probabilities /= np.sum(probabilities)

                next_city = np.random.choice(unvisited, p=probabilities)
                path.append(next_city)
                # increase the cost of move through the path
                path_length += calculate_distance(
                    cities[current_city], cities[next_city]
                )
                visited[next_city] = True
                # move to the next city, for the next iteration
                current_city = next_city

            paths.append(path)
            path_lengths.append(path_length)

            # update with current best path, this is a minimization problem
            if path_length < best_path_length:
                best_path = path
                best_path_length = path_length

        # remove a bit of pheromone of all map, it's a way to avoid local minima
        pheromone *= evaporation_rate

        # current ant must add pheromone to the path it has walked
        for path, path_length in zip(paths, path_lengths):
            for i in range(number_cities - 1):
                pheromone[path[i], path[i + 1]] += Q / path_length
            pheromone[path[-1], path[0]] += Q / path_length
    return best_path, best_path_length

Once we have basic functions related to _cities_ and _aco_, it is moment to run the algorithm creating first the list of cities and then calculating the best path.

In [ ]:
# model parameters
number_cities = 30
number_ants = 100
number_iterations = 100
alpha = 1
beta = 1
evaporation_rate = 0.1
Q = 4

# HERE create list of cities
cities = generate_cities(number_cities)

# HERE call ant_colony_optimization function
best_path, best_path_length = ant_colony_optimization(cities,number_ants,number_iterations,alpha,beta,evaporation_rate,Q)
best_path_list = list(best_path)


It is important to generate great visualizations in order to make simple your analysis. For this purpose, for any experiment the idea is to make a $3D$ plot (in this case using _matplotlib_) and print some metrics.

In [ ]:
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


def random_color() -> list:
    """
    This function generates a random color in RGB porcentage intensity.

    Returns:
    - list: A list with three random values between 0 and 1.
    """
    return [random.random(), random.random(), random.random()]



def plot_aco_route(cities: np.array, best_path: list):
    """
    This function plots the cities and the best path found by the ACO algorithm.

    Parameters:
    - cities (np.array): A list of cities with their coordinates.
    - best_path (list): The best path found by the ACO algorithm.
    """
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection="3d")

    for i in range(len(best_path) - 1):
        ax.plot(
            [cities[best_path[i], 0], cities[best_path[i + 1], 0]],  # x axis
            [cities[best_path[i], 1], cities[best_path[i + 1], 1]],  # y axis
            [cities[best_path[i], 2], cities[best_path[i + 1], 2]],  # z axis
            c=random_color(),
            linestyle="-",
            linewidth=3,
        )

    ax.plot(
        [cities[best_path[0], 0], cities[best_path[-1], 0]],
        [cities[best_path[0], 1], cities[best_path[-1], 1]],
        [cities[best_path[0], 2], cities[best_path[-1], 2]],
        c=random_color(),
        linestyle="-",
        linewidth=3,
    )

    ax.scatter(cities[0, 0], cities[0, 1], cities[0, 2], c="b", marker="o")
    ax.scatter(cities[1:, 0], cities[1:, 1], cities[1:, 2], c="g", marker="o")

    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_zlabel("Altitude")

    plt.show()


print("Best path:", best_path_list)
print("Best path length:", best_path_length)
plot_aco_route(np.array(cities), best_path_list)

